In [12]:
import re

# Функции для парсинга входных данных и построения доказательства
def parse_input(input_data):
    lines = input_data.strip().split("\n")
    context, expression = lines[0].split("|-")
    context = context.strip()
    expression = expression.strip()
    proof = [line.strip() for line in lines[1:]]
    return context, expression, proof

def add_brackets(expression):
    """Добавляет скобки вокруг выражения, если они отсутствуют."""
    if expression.count('->') + expression.count('&') + expression.count('|') <= 1:
        return expression
    return f"({expression})"

def transform_step(context, expression):
    # If the expression is part of the context, it can be directly used as a proof step
    if expression in context.split(','):
        return [expression]

    # Handling the context in the expression
    if context in expression and not any(op in context for op in ["&", "|", "->"]):
        return [context, add_brackets(expression)]
    
    # Handling conjunction (A&B) in the context
    if "&" in context:
        A, B = context.split("&")
        A, B = A.strip(), B.strip()
        
        # Decomposition of conjunction
        if expression in [A, B]:
            return [f"{context}->{add_brackets(expression)}", add_brackets(expression)]
        
        # Creation of conjunction
        if expression == f"{A}&{B}":
            return [f"{A}->{B}->{add_brackets(expression)}", f"{B}->{add_brackets(expression)}", add_brackets(expression)]
    
    # Handling disjunction (A|B) in the expression
    if "|" in expression:
        A, B = expression.split("|")
        A, B = A.strip(), B.strip()
        
        # Addition of disjunction
        if context == A or context == B:
            return [f"{context}->{expression}", expression]
        
    # If no transformation is applied, return the expression as it is
    return [add_brackets(expression)]

def build_proof(context, expression, proof):
    output = [f"{context}|-{expression}"]
    for step in proof:
        # Extract the context and expression from the step
        if "|-" in step:
            step_context, step_expression = step.split("|-")
            step_context = step_context.strip()
            step_expression = step_expression.strip()
        else:
            step_context = context
            step_expression = step
        
        # Transform the step using its context and expression
        transformed_step = transform_step(step_context, step_expression)
        output.extend(transformed_step)
    
    return "\n".join(output)

# Тестирование функции с обновленными шагами доказательства
input_data = """
|-A&B->A 
|-A&B->B 
A&B|-A
A&B|-B
A&B|-B->A->B&A
A&B|-A->B&A
A&B|-B&A
|-A&B->B&A
"""

context, expression, proof = parse_input(input_data)
output_data = build_proof(context, expression, proof)
print(output_data)

|-A&B->A

(A&B->B)
A&B->A
A
A&B->B
B
(B->A->B&A)
(A->B&A)
B&A


(A&B->B&A)
